In [1]:
# TEXT GENERATION - Run this AFTER training completes
import torch
from model import DeepSeekV3Config, DeepSeekV3ForCausalLM
import tiktoken
import os

# Check if checkpoint exists
checkpoint_path = "checkpoint_5500.pt"

if not os.path.exists(checkpoint_path):
    print("=" * 70)
    print("⚠️  ERROR: No trained model found!")
    print("=" * 70)
    print(f"\nThe checkpoint file '{checkpoint_path}' does not exist.")
    print("\nPlease run the training cell FIRST to create the model.")
    print("\nThe training cell will:")
    print("  1. Create and train DeepSeek-V3 from scratch")
    print("  2. Save the trained model to 'checkpoint_5500.pt'")
    print("  3. Then you can run this generation cell")
    print("\n" + "=" * 70)
else:
    # Load the trained model
    print("Loading trained model...")
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    config = checkpoint['config']
    model = DeepSeekV3ForCausalLM(config)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Device detection
    if torch.cuda.is_available():
        device = 'cuda'
        print(f"Using device: {device} ({torch.cuda.get_device_name(0)})")
    elif torch.backends.mps.is_available():
        device = 'mps'
        print(f"Using device: {device} (Apple Silicon GPU)")
    else:
        device = 'cpu'
        print(f"Using device: {device}")

    model = model.to(device)
    model.eval()

    print(f"\n✅ Model loaded successfully!")
    print(f"Parameters: {checkpoint['total_params']:,}")

    # Tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Generate text
    def generate_text(prompt_text, max_new_tokens=100, temperature=0.8, top_k=50):
        """Generate text from a prompt"""
        # Encode the prompt
        input_ids = tokenizer.encode(prompt_text)
        input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)
        
        print(f"\nPrompt: '{prompt_text}'")
        print(f"Generating {max_new_tokens} tokens...")
        print("-" * 70)
        
        # Generate
        with torch.no_grad():
            generated = model.generate(
                input_ids=input_ids,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k
            )
        
        # Decode
        generated_text = tokenizer.decode(generated[0].cpu().tolist())
        print(generated_text)
        print("-" * 70)
        return generated_text

    # Example generations
    print("\n" + "=" * 70)
    print("TEXT GENERATION EXAMPLES")
    print("=" * 70)

    # Example 1
    generate_text("Once upon a time", max_new_tokens=50, temperature=0.8)

    # Example 2
    generate_text("The meaning of life is", max_new_tokens=50, temperature=0.7)

    # Example 3 - Custom prompt (uncomment to use)
    # generate_text("Your custom prompt here", max_new_tokens=100, temperature=0.8)

⚠️  ERROR: No trained model found!

The checkpoint file 'checkpoint_5500.pt' does not exist.

Please run the training cell FIRST to create the model.

The training cell will:
  1. Create and train DeepSeek-V3 from scratch
  2. Save the trained model to 'checkpoint_5500.pt'
  3. Then you can run this generation cell



# PHASE 1: Train for 5000 steps and save checkpoint

Run this cell first. It will:
- Train your model for exactly 5000 steps
- Automatically stop after 5000 steps
- Save a checkpoint to `checkpoint_5000.pt`

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken
from tqdm import tqdm
import math

# Import DeepSeekV3 from model.py
from model import DeepSeekV3Config, DeepSeekV3ForCausalLM


# Dataset
class TextDataset(Dataset):
    def __init__(self, filepath, tokenizer, block_size=1024):
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.tokens = tokenizer.encode(text)

    def __len__(self):
        return len(self.tokens) - self.block_size

    def __getitem__(self, idx):
        input_ids = self.tokens[idx:idx + self.block_size]
        target_ids = self.tokens[idx + 1:idx + self.block_size + 1]
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)


# ==========================================
# PHASE 1: TRAIN FOR 5000 STEPS (Plain PyTorch)
# ==========================================

print("=" * 70)
print("PHASE 1: TRAINING FOR 5000 STEPS (Plain PyTorch)")
print("=" * 70)

# Configuration
USE_SMALL_MODEL = False  # Set to False to use full 135M model

if USE_SMALL_MODEL:
    print("\n[Using SMALL model]")
    config = DeepSeek-V3Config(
        vocab_size=50257,
        hidden_size=384,
        intermediate_size=1024,
        num_hidden_layers=12,
        num_attention_heads=6,
        num_key_value_heads=2,
        max_position_embeddings=1024,
        rms_norm_eps=1e-5,
        rope_theta=10000,
        tie_word_embeddings=True,
        attention_dropout=0.0,
    )
    block_size = 512
    batch_size = 4
    max_lr = 3e-4
    accumulate_grad_batches = 4
else:
    print("\n[Using FULL DeepSeek-V3-135M architecture]")
    config = DeepSeekV3Config(
        vocab_size=50257,
        hidden_size=576,
        num_hidden_layers=30,
        num_attention_heads=9,
        kv_lora_rank=512,
        moe_intermediate_size=256,
        n_shared_experts=1,
        n_routed_experts=8,
        num_experts_per_tok=2,
        max_position_embeddings=2048,
        rms_norm_eps=1e-5,
        rope_theta=10000.0,
        tie_word_embeddings=True,
        attention_dropout=0.0,
    )
    block_size = 1024
    batch_size = 2
    max_lr = 1e-3
    accumulate_grad_batches = 8

# Training parameters
warmup_steps = 100
max_steps = 5000  # Will stop at EXACTLY 5000 steps
log_interval = 50  # Print every 50 steps to avoid output overflow

# Device setup
if torch.cuda.is_available():
    device = 'cuda'
    print(f"\nUsing device: {device} ({torch.cuda.get_device_name(0)})")
elif torch.backends.mps.is_available():
    device = 'mps'
    print(f"\nUsing device: {device} (Apple Silicon GPU)")
else:
    device = 'cpu'
    print(f"\nUsing device: {device}")

# Setup
tokenizer = tiktoken.get_encoding("gpt2")

# Dataset and DataLoader
dataset = TextDataset("input-1.txt", tokenizer, block_size)
dataloader = DataLoader(
    dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0,
    pin_memory=False
)

# Model
model = DeepSeekV3ForCausalLM(config)
model = model.to(device)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel Configuration:")
print(f"  - Architecture: DeepSeek-V3 (MLA + MoE)")
print(f"  - Vocabulary size: {config.vocab_size:,}")
print(f"  - Hidden size: {config.hidden_size}")
print(f"  - Layers: {config.num_hidden_layers}")
print(f"  - Attention heads: {config.num_attention_heads}")
print(f"  - KV LoRA Rank: {config.kv_lora_rank}")
print(f"  - MoE Experts: 1 Shared + {config.n_routed_experts} Routed")
print(f"  - Total parameters: {total_params:,}")

print(f"\nTraining Configuration (Phase 1):")
print(f"  - Max steps: {max_steps}")
print(f"  - Batch size: {batch_size}")
print(f"  - Gradient accumulation: {accumulate_grad_batches}")
print(f"  - Effective batch size: {batch_size * accumulate_grad_batches}")
print(f"  - Warmup steps: {warmup_steps}")
print(f"  - Max learning rate: {max_lr}")
print(f"  - Logging interval: Every {log_interval} steps")

# Optimizer - separate decay parameters
decay_params = []
no_decay_params = []

for name, param in model.named_parameters():
    if param.requires_grad:
        if 'bias' in name or 'norm' in name or 'embed' in name:
            no_decay_params.append(param)
        else:
            decay_params.append(param)

optimizer = torch.optim.AdamW([
    {'params': decay_params, 'weight_decay': 0.01},
    {'params': no_decay_params, 'weight_decay': 0.0}
], lr=max_lr, betas=(0.9, 0.95), eps=1e-8)

# Learning rate scheduler
def get_lr(step, warmup_steps, max_steps, max_lr):
    if step < warmup_steps:
        return max_lr * (step / warmup_steps)
    else:
        progress = (step - warmup_steps) / (max_steps - warmup_steps)
        return max_lr * max(0.1, 1.0 - progress)

# Clear cache
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
elif torch.cuda.is_available():
    torch.cuda.empty_cache()

torch.set_float32_matmul_precision('high')

# ==========================================
# INITIAL LOSS CHECK
# ==========================================
print("\n" + "=" * 70)
print("INITIAL LOSS CHECK (before training)")
print("=" * 70)

model.eval()
with torch.no_grad():
    # Get a sample batch
    sample_batch = next(iter(dataloader))
    input_ids, target_ids = sample_batch
    input_ids = input_ids.to(device)
    target_ids = target_ids.to(device)
    
    # Forward pass
    outputs = model(input_ids=input_ids, labels=target_ids)
    initial_loss = outputs['loss'].item()
    
    print(f"\nInitial loss (random weights): {initial_loss:.4f}")
    print(f"Expected loss for random model: ~{math.log(config.vocab_size):.2f}")
    
    if initial_loss > 50:
        print(f"\n⚠️  WARNING: Loss is unusually high ({initial_loss:.4f})!")
        print("This suggests a potential issue with loss calculation.")
    elif 8 < initial_loss < 15:
        print(f"\n✅ Loss is in expected range for random initialization!")
    else:
        print(f"\n⚠️  Loss is {initial_loss:.4f}, which is outside typical range (8-15)")

print("=" * 70 + "\n")
model.train()

# ==========================================
# TRAINING LOOP
# ==========================================
print("=" * 70)
print("STARTING PHASE 1 TRAINING")
print(f"Will stop at EXACTLY {max_steps} steps")
print(f"Logging every {log_interval} steps")
print("=" * 70 + "\n")

# Training loop
update_step = 0  # Actual optimizer update steps
batch_idx = 0    # Batch counter for gradient accumulation
accumulated_loss = 0.0
optimizer.zero_grad()

# Create infinite dataloader
def cycle(dataloader):
    while True:
        for batch in dataloader:
            yield batch

data_iter = cycle(dataloader)

# Progress bar
pbar = tqdm(total=max_steps, desc="Training", unit="step")

while update_step < max_steps:
    # Get batch
    input_ids, target_ids = next(data_iter)
    input_ids = input_ids.to(device)
    target_ids = target_ids.to(device)

    # Forward pass
    outputs = model(input_ids=input_ids, labels=target_ids)
    loss = outputs['loss']

    # Scale loss for gradient accumulation
    loss = loss / accumulate_grad_batches
    loss.backward()

    # Accumulate the UNSCALED loss for logging (multiply back)
    accumulated_loss += loss.item() * accumulate_grad_batches
    batch_idx += 1

    # Update weights after accumulation
    if batch_idx % accumulate_grad_batches == 0:
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update learning rate
        lr = get_lr(update_step, warmup_steps, max_steps, max_lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # Optimizer step
        optimizer.step()
        optimizer.zero_grad()

        # Calculate average loss over accumulated batches
        avg_loss = accumulated_loss / accumulate_grad_batches
        
        # Update progress bar every step
        pbar.set_postfix({'loss': f'{avg_loss:.4f}', 'lr': f'{lr:.6f}'})
        
        # Print only at log_interval to avoid output overflow
        if (update_step + 1) % log_interval == 0 or update_step == 0:
            print(f"Step {update_step + 1}/{max_steps} | Loss: {avg_loss:.4f} | LR: {lr:.6f}")
        
        accumulated_loss = 0.0

        pbar.update(1)
        update_step += 1
        batch_idx = 0  # Reset batch counter

        # EXACT STOP at max_steps
        if update_step >= max_steps:
            print(f"\n✅ Reached {max_steps} steps - stopping training!")
            break

pbar.close()

print("\n" + "=" * 70)
print("PHASE 1 COMPLETED!")
print("=" * 70)

# Save checkpoint
checkpoint_path = "checkpoint_5000.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'config': config,
    'total_params': total_params,
    'global_step': update_step,
}, checkpoint_path)

print(f"\n✅ Checkpoint saved to: {checkpoint_path}")
print(f"✅ Trained for exactly {update_step} steps")
print(f"✅ Final loss: {avg_loss:.4f}")
print("\nYou can now run PHASE 2 to continue training for 500 more steps!")

# Cleanup
del model, optimizer
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
elif torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Memory cleared.")

PHASE 1: TRAINING FOR 5000 STEPS (Plain PyTorch)

[Using FULL DeepSeek-V3-135M architecture]

Using device: mps (Apple Silicon GPU)

Model Configuration:
  - Architecture: DeepSeek-V3 (MLA + MoE)
  - Vocabulary size: 50,257
  - Hidden size: 576
  - Layers: 30
  - Attention heads: 9
  - KV LoRA Rank: 512
  - MoE Experts: 1 Shared + 8 Routed
  - Total parameters: 174,012,288

Training Configuration (Phase 1):
  - Max steps: 5000
  - Batch size: 2
  - Gradient accumulation: 8
  - Effective batch size: 16
  - Warmup steps: 100
  - Max learning rate: 0.001
  - Logging interval: Every 50 steps

INITIAL LOSS CHECK (before training)

Initial loss (random weights): 10.9710
Expected loss for random model: ~10.82

✅ Loss is in expected range for random initialization!

STARTING PHASE 1 TRAINING
Will stop at EXACTLY 5000 steps
Logging every 50 steps



Training:   0%|          | 1/5000 [00:35<48:47:38, 35.14s/step, loss=11.1523, lr=0.000000]

Step 1/5000 | Loss: 11.1523 | LR: 0.000000


Training:   1%|          | 50/5000 [12:14<17:04:33, 12.42s/step, loss=6.3426, lr=0.000490]

Step 50/5000 | Loss: 6.3426 | LR: 0.000490


...

Step 4900/5000 | Loss: 0.0648 | LR: 0.000100


Training:  99%|█████████▉| 4950/5000 [23:41:06<11:32, 13.85s/step, loss=0.0621, lr=0.000100]

Step 4950/5000 | Loss: 0.0621 | LR: 0.000100


Training: 100%|██████████| 5000/5000 [23:52:37<00:00, 17.19s/step, loss=0.0652, lr=0.000100]


Step 5000/5000 | Loss: 0.0652 | LR: 0.000100

✅ Reached 5000 steps - stopping training!

PHASE 1 COMPLETED!

✅ Checkpoint saved to: checkpoint_5000.pt
✅ Trained for exactly 5000 steps
✅ Final loss: 0.0652

You can now run PHASE 2 to continue training for 500 more steps!
Memory cleared.


# PHASE 2: Load checkpoint and train for 500 more steps

Run this cell after Phase 1 completes. It will:
- Load the checkpoint from `checkpoint_5000.pt`
- Resume training from step 5000
- Train for 500 more steps (total: 5500 steps)
- Save final checkpoint to `checkpoint_5500.pt`

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken
from tqdm import tqdm
import math
import os

# Import DeepSeekV3 from model.py
from model import DeepSeekV3Config, DeepSeekV3ForCausalLM


# Dataset
class TextDataset(Dataset):
    def __init__(self, filepath, tokenizer, block_size=1024):
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.tokens = tokenizer.encode(text)

    def __len__(self):
        return len(self.tokens) - self.block_size

    def __getitem__(self, idx):
        input_ids = self.tokens[idx:idx + self.block_size]
        target_ids = self.tokens[idx + 1:idx + self.block_size + 1]
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)


# ==========================================
# PHASE 2: LOAD CHECKPOINT AND TRAIN 500 MORE STEPS (Plain PyTorch)
# ==========================================

print("=" * 70)
print("PHASE 2: LOADING CHECKPOINT AND TRAINING 500 MORE STEPS")
print("=" * 70)

# Check if checkpoint exists
checkpoint_path = "checkpoint_5000.pt"
if not os.path.exists(checkpoint_path):
    print(f"\n❌ ERROR: Checkpoint not found at {checkpoint_path}")
    print("Please run PHASE 1 first to create the checkpoint!")
    raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

# Load checkpoint
print(f"\n📂 Loading checkpoint from: {checkpoint_path}")
# Note: weights_only=False is safe here because we created this checkpoint ourselves
checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

config = checkpoint['config']
previous_step = checkpoint['global_step']

print(f"✅ Checkpoint loaded successfully!")
print(f"   Previous training: {previous_step} steps")

# Configuration
USE_SMALL_MODEL = False  # Should match Phase 1

if USE_SMALL_MODEL:
    block_size = 512
    batch_size = 4
    max_lr = 3e-4
    accumulate_grad_batches = 4
else:
    block_size = 1024
    batch_size = 2
    max_lr = 1e-3
    accumulate_grad_batches = 8

# Training parameters for PHASE 2
warmup_steps = 100  # Already completed in Phase 1
additional_steps = 500  # Train for 500 MORE steps
new_max_steps = previous_step + additional_steps  # Total: 5500 steps
log_interval = 50  # Print every 50 steps to avoid output overflow

print(f"\nPhase 2 Training Plan:")
print(f"  - Starting from step: {previous_step}")
print(f"  - Training for: {additional_steps} more steps")
print(f"  - Final step will be: {new_max_steps}")
print(f"  - Logging interval: Every {log_interval} steps")

# Device setup
if torch.cuda.is_available():
    device = 'cuda'
    print(f"\nUsing device: {device} ({torch.cuda.get_device_name(0)})")
elif torch.backends.mps.is_available():
    device = 'mps'
    print(f"\nUsing device: {device} (Apple Silicon GPU)")
else:
    device = 'cpu'
    print(f"\nUsing device: {device}")

# Setup
tokenizer = tiktoken.get_encoding("gpt2")

# Dataset and DataLoader
dataset = TextDataset("input-1.txt", tokenizer, block_size)
dataloader = DataLoader(
    dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0,
    pin_memory=False
)

# Model
model = DeepSeekV3ForCausalLM(config)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

print(f"\n✅ Model weights loaded from checkpoint")

total_params = sum(p.numel() for p in model.parameters())
print(f"   Total parameters: {total_params:,}")

# Optimizer - separate decay parameters
decay_params = []
no_decay_params = []

for name, param in model.named_parameters():
    if param.requires_grad:
        if 'bias' in name or 'norm' in name or 'embed' in name:
            no_decay_params.append(param)
        else:
            decay_params.append(param)

optimizer = torch.optim.AdamW([
    {'params': decay_params, 'weight_decay': 0.01},
    {'params': no_decay_params, 'weight_decay': 0.0}
], lr=max_lr, betas=(0.9, 0.95), eps=1e-8)

# Load optimizer state
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

print("✅ Optimizer state restored!")

# Learning rate scheduler
def get_lr(step, warmup_steps, max_steps, max_lr):
    if step < warmup_steps:
        return max_lr * (step / warmup_steps)
    else:
        progress = (step - warmup_steps) / (max_steps - warmup_steps)
        return max_lr * max(0.1, 1.0 - progress)

# Clear cache
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
elif torch.cuda.is_available():
    torch.cuda.empty_cache()

torch.set_float32_matmul_precision('high')

print("\n" + "=" * 70)
print("STARTING PHASE 2 TRAINING")
print(f"Resuming from step {previous_step}, training to step {new_max_steps}")
print(f"Will stop at EXACTLY {new_max_steps} steps")
print("=" * 70 + "\n")

# Training loop
model.train()
update_step = previous_step  # Start from where we left off
batch_idx = 0  # Batch counter for gradient accumulation
accumulated_loss = 0.0
optimizer.zero_grad()

# Create infinite dataloader
def cycle(dataloader):
    while True:
        for batch in dataloader:
            yield batch

data_iter = cycle(dataloader)

# Progress bar
pbar = tqdm(total=new_max_steps, initial=previous_step, desc="Training", unit="step")

while update_step < new_max_steps:
    # Get batch
    input_ids, target_ids = next(data_iter)
    input_ids = input_ids.to(device)
    target_ids = target_ids.to(device)

    # Forward pass
    outputs = model(input_ids=input_ids, labels=target_ids)
    loss = outputs['loss']

    # Scale loss for gradient accumulation
    loss = loss / accumulate_grad_batches
    loss.backward()

    # Accumulate the UNSCALED loss for logging (multiply back)
    accumulated_loss += loss.item() * accumulate_grad_batches
    batch_idx += 1

    # Update weights after accumulation
    if batch_idx % accumulate_grad_batches == 0:
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update learning rate
        lr = get_lr(update_step, warmup_steps, new_max_steps, max_lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # Optimizer step
        optimizer.step()
        optimizer.zero_grad()

        # Calculate average loss over accumulated batches
        avg_loss = accumulated_loss / accumulate_grad_batches
        
        # Update progress bar every step
        pbar.set_postfix({'loss': f'{avg_loss:.4f}', 'lr': f'{lr:.6f}'})
        
        # Print only at log_interval to avoid output overflow
        if (update_step + 1) % log_interval == 0 or update_step == previous_step:
            print(f"Step {update_step + 1}/{new_max_steps} | Loss: {avg_loss:.4f} | LR: {lr:.6f}")
        
        accumulated_loss = 0.0

        pbar.update(1)
        update_step += 1
        batch_idx = 0  # Reset batch counter

        # EXACT STOP at new_max_steps
        if update_step >= new_max_steps:
            print(f"\n✅ Reached {new_max_steps} steps - stopping training!")
            break

pbar.close()

print("\n" + "=" * 70)
print("PHASE 2 COMPLETED!")
print("=" * 70)

# Save final checkpoint
final_checkpoint_path = "checkpoint_5500.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'config': config,
    'total_params': total_params,
    'global_step': update_step,
}, final_checkpoint_path)

print(f"\n✅ Final checkpoint saved to: {final_checkpoint_path}")
print(f"✅ Total training steps: {update_step}")
print(f"✅ Final loss: {avg_loss:.4f}")
print(f"\n🎉 Training complete! Model trained for {previous_step} + {additional_steps} = {update_step} steps")

# Cleanup
del model, optimizer
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
elif torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Memory cleared.")

PHASE 2: LOADING CHECKPOINT AND TRAINING 500 MORE STEPS

📂 Loading checkpoint from: checkpoint_5000.pt
✅ Checkpoint loaded successfully!
   Previous training: 5000 steps

Phase 2 Training Plan:
  - Starting from step: 5000
  - Training for: 500 more steps
  - Final step will be: 5500
  - Logging interval: Every 50 steps

Using device: mps (Apple Silicon GPU)

✅ Model weights loaded from checkpoint
   Total parameters: 174,012,288
✅ Optimizer state restored!

STARTING PHASE 2 TRAINING
Resuming from step 5000, training to step 5500
Will stop at EXACTLY 5500 steps



Training:  91%|█████████ | 5001/5500 [00:31<4:25:49, 31.96s/step, loss=0.0647, lr=0.000100]

Step 5001/5500 | Loss: 0.0647 | LR: 0.000100


Training:  92%|█████████▏| 5050/5500 [12:28<1:50:14, 14.70s/step, loss=0.0632, lr=0.000100]

Step 5050/5500 | Loss: 0.0632 | LR: 0.000100


...

Step 5400/5500 | Loss: 0.0641 | LR: 0.000100


Training:  99%|█████████▉| 5450/5500 [1:48:56<12:05, 14.52s/step, loss=0.0574, lr=0.000100]

Step 5450/5500 | Loss: 0.0574 | LR: 0.000100


Training: 100%|██████████| 5500/5500 [2:00:59<00:00, 14.52s/step, loss=0.0609, lr=0.000100]


Step 5500/5500 | Loss: 0.0609 | LR: 0.000100

✅ Reached 5500 steps - stopping training!

PHASE 2 COMPLETED!

✅ Final checkpoint saved to: checkpoint_5500.pt
✅ Total training steps: 5500
✅ Final loss: 0.0609

🎉 Training complete! Model trained for 5000 + 500 = 5500 steps
Memory cleared.
